<a href="https://colab.research.google.com/github/Pedro4010/caseifood/blob/main/caseifood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Ifood

### O objetivo desse trabalho é desenvolver um modelo preditivo que vai produzir maior retorno para a próxima campanha de marketing do Ifood, além de entender as características dos clientes que estão dispostos a comprar o produto.

### Esse notebook foi usado para limpeza e pré processamento dos dados, identificação de alguns erros que poderiam atrapalhar as análises.

In [ ]:
# Conectando ao drive e lendo o csv através do pandas.
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/googlecolab/ml_project1_data.csv', index_col = 'ID')

Mounted at /content/drive


##### O conjunto de dados contém características sociodemográficas e firmográficas de cerca de 2.240 clientes que foram contatados. Além disso, contém um sinalizador para os clientes que responderam à campanha.


**Feature** -> **Descrição**
*   AcceptedCmp1 -> 1 se o cliente aceitou a oferta na 1st campanha, 0 se não.
*   AcceptedCmp2 -> 1 se o cliente aceitou a oferta na 2st campanha, 0 se não.
*   AcceptedCmp3 -> 1 se o cliente aceitou a oferta na 3st campanha, 0 se não.
*   AcceptedCmp4 -> 1 se o cliente aceitou a oferta na 4st campanha, 0 se não.
*   AcceptedCmp5 -> 1 se o cliente aceitou a oferta na 5st campanha, 0 se não.
*   Response(target) -> 1 se o cliente aceitou a oferta na última campanha, 0 se não.
*   Complain -> 1 se o cliente reclamou nos últimos 2 anos
*   DtCustomer -> dados de cadastro do cliente na empresa
*   Education -> nível de instrução do cliente
*   Marital -> estado civil do cliente
*   Kidhome -> número de crianças pequenas na casa do cliente
*   Teenhome -> número de adolescentes na casa do cliente
*   Income -> renda familiar anual do cliente
*   MntFishProducts -> valor gasto em produtos de peixe nos últimos 2 anos 
*   MntMeatProducts -> valor gasto em produtos de carne nos últimos 2 anos
*   MntFruits -> valor gasto em produtos de frutas nos últimos 2 anos
*   MntSweetProducts -> valor gasto em produtos doces nos últimos 2 anos
*   MntWines -> valor gasto em produtos vitivinícolas nos últimos 2 anos
*   MntGoldProds -> valor gasto em produtos de ouro (especiais) nos últimos 2 anos
*   NumDealsPurchases -> número de compras feitas com desconto
*   NunCatalogPurchases -> número de compras feitas usando o catálogo
*   NunStorePurchases -> número de compras feitas diretamente nas lojas
*   NumWebPurchases -> número de compras feitas através do site da empresa 
*   NumWebVisitsMonth -> número de visitas ao site da empresa no último mês
*   Recency -> número de dias desde a última compra    

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

In [ ]:
df.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
ID,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,...,7,0,0,0,0,0,0,3,11,1
2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,...,5,0,0,0,0,0,0,3,11,0
4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,...,4,0,0,0,0,0,0,3,11,0
6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,...,6,0,0,0,0,0,0,3,11,0
5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,...,5,0,0,0,0,0,0,3,11,0


##### Em um primeiro olhar sobre o DataSet, optei por já eliminar algumas colunas que não fazem sentido em um primeiro momento. Dependendo do resultado das análises, talvez eu volte e acrescente-as.
##### A coluna ID do cliente se tornou meu índice, apenas um número de cadastro. Também removi as colunas Z_CostContact e Z_Revenue, visto que ambas possuem os mesmos valores por todo o DataSet.

In [ ]:
# Verificando se todos os valores são iguais.
df['Z_CostContact'].value_counts()

3    2240
Name: Z_CostContact, dtype: int64

In [ ]:
# Verificando se todos os valores são iguais.
df['Z_Revenue'].value_counts()

11    2240
Name: Z_Revenue, dtype: int64

In [ ]:
dfi = df.drop(columns=['Z_CostContact','Z_Revenue'])

In [ ]:
# Conferindo se as colunas foram removidas.
dfi.columns

Index(['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Response'],
      dtype='object')

In [ ]:
# Verificando se há valores nulos. Como podemos ver, temos 24 valores nulos em Income. Vamos utilizar uma média para tratar esses valores mais abaixo.
# Em alguns casos, seria mais adequado uma regressão linear, por exemplo, mas neste caso, a média e a mediana estão bem próximas, e isso é um fator relevante.
dfi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2240 entries, 5524 to 9405
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year_Birth           2240 non-null   int64  
 1   Education            2240 non-null   object 
 2   Marital_Status       2240 non-null   object 
 3   Income               2216 non-null   float64
 4   Kidhome              2240 non-null   int64  
 5   Teenhome             2240 non-null   int64  
 6   Dt_Customer          2240 non-null   object 
 7   Recency              2240 non-null   int64  
 8   MntWines             2240 non-null   int64  
 9   MntFruits            2240 non-null   int64  
 10  MntMeatProducts      2240 non-null   int64  
 11  MntFishProducts      2240 non-null   int64  
 12  MntSweetProducts     2240 non-null   int64  
 13  MntGoldProds         2240 non-null   int64  
 14  NumDealsPurchases    2240 non-null   int64  
 15  NumWebPurchases      2240 non-null 

In [ ]:
# Entendendo um pouco mais como estão distribuidos os dados.
dfi.describe().T

,count,mean,std,min,25%,50%,75%,max
Year_Birth,2240.0,1968.805804,11.984069,1893.0,1959.00,1970.0,1977.00,1996.0
Income,2216.0,52247.251354,25173.076661,1730.0,35303.00,51381.5,68522.00,666666.0
Kidhome,2240.0,0.444196,0.538398,0.0,0.00,0.0,1.00,2.0
Teenhome,2240.0,0.506250,0.544538,0.0,0.00,0.0,1.00,2.0
Recency,2240.0,49.109375,28.962453,0.0,24.00,49.0,74.00,99.0
MntWines,2240.0,303.935714,336.597393,0.0,23.75,173.5,504.25,1493.0
MntFruits,2240.0,26.302232,39.773434,0.0,1.00,8.0,33.00,199.0
MntMeatProducts,2240.0,166.950000,225.715373,0.0,16.00,67.0,232.00,1725.0
MntFishProducts,2240.0,37.525446,54.628979,0.0,3.00,12.0,50.00,259.0
MntSweetProducts,2240.0,27.062946,41.280498,0.0,1.00,8.0,33.00,263.0


In [ ]:
# Verificando um possível outlier na coluna Income. 
# Transformar em um valor nulo e corrigir junto com os outros valores nulos
# dfi[dfi['Income']==666666]. Foi verificado que é um outlier.
dfi['Income'] = dfi['Income'].replace(666666, np.nan)

In [ ]:
# O valor foi corretamente substituído, como podemos ver abaixo.
# Antes tínhamos 24 nulos e agora temos 25.
dfi.isnull().sum()

Year_Birth              0
Education               0
Marital_Status          0
Income                 25
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Response                0
dtype: int64

In [ ]:
# Verificando a média e a mediana da coluna.
dfi['Income'].describe()

count      2215.000000
mean      51969.861400
std       21526.320095
min        1730.000000
25%       35284.000000
50%       51373.000000
75%       68487.000000
max      162397.000000
Name: Income, dtype: float64

In [ ]:
# Substituindo os valores nulos pela média.
dfi['Income'] = dfi['Income'].replace(np.nan, dfi['Income'].mean())

In [ ]:
# Rodamos o somatório de nulos novamente e a operação foi realizada. Não há mais valores nulos em dfi.
dfi.isnull().sum()

Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
Response               0
dtype: int64

In [ ]:
# Criando uma coluna com o número de pessoas que aderiram a campanha.

dfi['Total_Accepted'] = df['AcceptedCmp1'] + dfi['AcceptedCmp2'] + dfi['AcceptedCmp3'] + dfi['AcceptedCmp4'] + dfi['AcceptedCmp5'] 

In [ ]:
# Contando os valores da nova coluna.
dfi['Total_Accepted'].value_counts()

0    1777
1     325
2      83
3      44
4      11
Name: Total_Accepted, dtype: int64

In [ ]:
# Transformando a coluna de Data.
dfi['Dt_Customer'] = pd.to_datetime(dfi['Dt_Customer'])

In [ ]:
# A coluna year_birth por si só pode não ser tão interessante pra nós, por isso optamos por calcular a idade do cliente.
dfi['Age'] = datetime.now().year - dfi['Year_Birth']

In [ ]:
dfi['Dt_Customer'].max()

Timestamp('2014-06-29 00:00:00')

In [ ]:
# Em vez de pegar a data de hoje e ver há quanto tempo o cliente está, vamos olhar pelo cliente mais novo para o mais antigo, 
# pois como é um dataset mais antigo, calcular até os dias de hoje poderia trazer algum problema.
dfi['Days_a_customer'] = dfi['Dt_Customer'].max() - dfi['Dt_Customer']

In [ ]:
# Fazendo com que fique somente os valores numéricos.
dfi['Days_a_customer'] = dfi['Days_a_customer'].apply(lambda x: x.days)

In [ ]:
dfi['Days_a_customer']

ID
5524     663
2174     113
4141     312
6182     139
5324     161
        ... 
10870    381
4001      19
7270     155
8235     156
9405     622
Name: Days_a_customer, Length: 2240, dtype: int64

In [ ]:
# Como tratar essa coluna ?
# Criar um map pra auxiliar.
# Fazer um dummy seria o ideal, porém vamos ver como fica com o map.
dfi['Marital_Status'].value_counts()

Married     864
Together    580
Single      480
Divorced    232
Widow        77
Alone         3
Absurd        2
YOLO          2
Name: Marital_Status, dtype: int64

In [ ]:
marital_map = {
    'Married': 2,
    'Together': 2,
    'Single': 1,
    'Divorced': 1,
    'Widow': 1,
    'Alone': 1,
    'Absurd': 1,
    'YOLO': 1
    
}

In [ ]:
dfi['Marital_Status_Custom'] = dfi['Marital_Status'].map(marital_map)

In [ ]:
dfi['Marital_Status_Custom'].value_counts()

2    1444
1     796
Name: Marital_Status_Custom, dtype: int64

In [ ]:
# Criando uma coluna para o total de crianças/adolescentes em casa.
dfi['Total_childs'] = dfi['Kidhome'] + dfi['Teenhome']

In [ ]:
# Criar uma coluna com o tamanho da família
dfi['Family_size'] = dfi['Total_childs'] + dfi['Marital_Status_Custom']

In [ ]:
# Criar uma coluna com o total geral comprado de cada cliente, independente do produto.
dfi['MntTotal']= dfi.iloc[:, 8:14].sum(axis=1)

In [ ]:
dfi.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Total_Accepted,Age,Days_a_customer,Marital_Status_Custom,Total_childs,Family_size,MntTotal
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,...,0,0,0,0,0,0,1,0,65,663,1,0,1,1617
2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,...,0,0,0,0,0,0,0,0,68,113,1,2,3,27
4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,...,0,0,0,0,0,0,0,0,57,312,2,0,2,776
6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,...,0,0,0,0,0,0,0,0,38,139,2,1,3,53
5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,...,0,0,0,0,0,0,0,0,41,161,2,1,3,422


In [ ]:
# Como tratamos algumas colunas, as antigas podem ser excluídas, como Year_Birth e Dt_Customer.
dfi.drop(columns=['Year_Birth','Dt_Customer'],inplace=True)

In [ ]:
customers = dfi.copy()

In [ ]:
# Dropando essas colunas também.
customers.drop(columns=['Marital_Status','Kidhome','Teenhome'],inplace=True)

In [ ]:
customers.head()

,Education,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Total_Accepted,Age,Days_a_customer,Marital_Status_Custom,Total_childs,Family_size,MntTotal
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,Graduation,58138.0,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,1,0,65,663,1,0,1,1617
2174,Graduation,46344.0,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,0,0,68,113,1,2,3,27
4141,Graduation,71613.0,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,0,0,57,312,2,0,2,776
6182,Graduation,26646.0,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,0,0,38,139,2,1,3,53
5324,PhD,58293.0,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,0,0,41,161,2,1,3,422


In [ ]:
# Transformar a coluna Education para numérico
# Fazer isso com dummy. Aqui escolaridade não tem um peso.
customers = pd.get_dummies(data=customers, columns=['Education'])
customers.head()

In [ ]:
customers.head()

,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Total_Accepted,Age,Days_a_customer,Marital_Status_Custom,Total_childs,Family_size,MntTotal,Education_2n Cycle,Education_Basic,Education_Graduation,Education_Master,Education_PhD
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,58138.0,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,1,0,65,663,1,0,1,1617,0,0,1,0,0
2174,46344.0,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,0,0,68,113,1,2,3,27,0,0,1,0,0
4141,71613.0,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,0,0,57,312,2,0,2,776,0,0,1,0,0
6182,26646.0,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,0,0,38,139,2,1,3,53,0,0,1,0,0
5324,58293.0,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,0,0,41,161,2,1,3,422,0,0,0,0,1


In [ ]:
# Usei esse comando apenas para verificar como estavam os dados das colunas que não apareciam com default.
pd.set_option('max_columns',32)

In [ ]:
# Tem que transformar as colunas para inteiro ? 
customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2240 entries, 5524 to 9405
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Income                 2240 non-null   float64
 1   Recency                2240 non-null   int64  
 2   MntWines               2240 non-null   int64  
 3   MntFruits              2240 non-null   int64  
 4   MntMeatProducts        2240 non-null   int64  
 5   MntFishProducts        2240 non-null   int64  
 6   MntSweetProducts       2240 non-null   int64  
 7   MntGoldProds           2240 non-null   int64  
 8   NumDealsPurchases      2240 non-null   int64  
 9   NumWebPurchases        2240 non-null   int64  
 10  NumCatalogPurchases    2240 non-null   int64  
 11  NumStorePurchases      2240 non-null   int64  
 12  NumWebVisitsMonth      2240 non-null   int64  
 13  AcceptedCmp3           2240 non-null   int64  
 14  AcceptedCmp4           2240 non-null   int64  
 15  A

In [ ]:
customers.to_csv('/content/drive/MyDrive/googlecolab/cleaned_ifood.csv')